In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ipyparallel as ipp
print(ipp.version_info)

nproc = 4
cluster = ipp.Cluster(engines='mpi', n=nproc, shutdown_atexit=False)
print(cluster)
client = cluster.start_and_connect_sync(activate=True)
view = client[:]
client.ids

(8, 6, 1)
<Cluster(cluster_id='1687877084-j6kq', profile='default')>
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.44s/engine]


[0, 1, 2, 3]

In [6]:
!ipcluster list

[stdout:2] PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687877084-j6kq                  True          4 MPI


[stdout:3] PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687877084-j6kq                  True          4 MPI


[stdout:0] PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687877084-j6kq                  True          4 MPI


[stdout:1] PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687877084-j6kq                  True          4 MPI


In [4]:
%autopx --block --group-outputs=engine

%autopx enabled


In [33]:
import logging
from pathlib import Path
from time import strftime

from mpi4py.MPI import COMM_WORLD
world_size = COMM_WORLD.Get_size()
world_rank = COMM_WORLD.Get_rank()

LOGGER_NAME = 'QTM'
LOGFILE_DIR = 'test.log'
LOG_FORMAT = f'%(asctime)s - proc {world_rank}/{world_size} - level %(levelno)s: %(message)s'


logger = logging.getLogger(LOGGER_NAME)
print(logger.hasHandlers())

logfile_path = Path(LOGFILE_DIR)
if logfile_path.is_file():
    COMM_WORLD.barrier()
    if world_rank == 0:
        logfile_path.rename(
            logfile_path.with_suffix(
                f'.pre{strftime("%Y%m%d-%H%M%S")}' + logfile_path.suffix
            ))
    

logger_file = logging.FileHandler(LOGFILE_DIR)
logger_file.setLevel(logging.INFO)

logger_fmt = logging.Formatter(LOG_FORMAT)

logger_file.setFormatter(logger_fmt)
logger.addHandler(logger_file)

warn_logger = logging.getLogger('py.warnings')
warn_logger.addHandler(logger_file)

[stdout:2] True


[stdout:1] True


[stdout:0] True


[stdout:3] True


In [34]:
from time import sleep

# logger.handlers.clear()
logger.setLevel(logging.DEBUG)
logger_file.setLevel(logging.INFO)
logger.info('Hello')
if world_rank == 1:
    sleep(2)
    logger.warning('woke up')
logger.debug('am here')

%px: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 444.42tasks/s]


In [35]:
from logging import captureWarnings
import warnings

captureWarnings(True)
warnings.warn("A Warning")


In [10]:
%autopx --block --group-outputs=engine

%autopx disabled


In [11]:
cluster.restart_engines_sync()
!ipcluster list

Stopping engine(s): 1687876361
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
engine set stopped 1687876361: {'exit_code': 1, 'pid': 10165, 'identifier': 'ipengine-1687876360-2zxj-1687876361-9810'}
PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687876360-2zxj                  True          4 MPI
